In [1]:
import pandas as pd
import pandas as pd
from unidecode import unidecode
import re
# URL to the index CSV file
url = 'https://raw.githubusercontent.com/gabriel1200/metric_test/refs/heads/master/index_master.csv'

# Read the CSV file into a DataFrame
index_df = pd.read_csv(url)
index_df=index_df[index_df.year>1990]
index_df=index_df[index_df.team!='TOT']
def remove_acronyms(player_name):
    if isinstance(player_name, str):  # Only process if player_name is a string
        # Define a regex pattern to remove common suffixes/acronyms
        pattern = r'\b(Jr\.|Sr\.| Jr| Sr|II|III|IV|V|VI)\b'
        # Remove accents and non-ASCII characters
        player_name = unidecode(player_name)
        # Use regex to substitute any suffix patterns
        cleaned_name = re.sub(pattern, '', player_name).strip()
        return cleaned_name
    else:
        return player_name  # If not a string (like NaN), return as-is

# Convert non-English characters in the 'player' column to their ASCII/English equivalent
index_df['player']=index_df['player'].astype(str)

index_df['player'] = index_df['player'].apply(lambda x: unidecode(str(x)))
# Display the cleaned 'player' column to verify the conversion
team_acronym_map = {
    'NJN': 'BKN',  # New Jersey Nets -> Brooklyn Nets
    'BRK':'BKN',
    'VAN': 'MEM',  # Vancouver Grizzlies -> Memphis Grizzlies
    'CHA': 'CHA',  # Old Charlotte Hornets -> Modern Charlotte Hornets
    'NOH': 'NOP',  # New Orleans Hornets -> New Orleans Pelicans
    'SEA': 'OKC',  # Seattle SuperSonics -> Oklahoma City Thunder
    'WSB': 'WAS',  # Washington Bullets -> Washington Wizards
    'CHH': 'CHA',  # Charlotte Hornets -> Modern Charlotte Hornets
    'CHO':'CHA',
    'SDC': 'LAC',  # San Diego Clippers -> LA Clippers
    'KCK': 'SAC',  # Kansas City Kings -> Sacramento Kings
    'NOK':'NOP',

    'PHO': 'PHX',
}

index_df['team'] = index_df['team'].replace(team_acronym_map)

# Save the cleaned index DataFrame if necessary

# Proceed with the merging step with the salary data, once the salary data is loaded into 'salary_df'
# Assuming 'salary_df' is already prepared from the salary scraping
# salary_df['Player'] = salary_df['Player'].apply(lambda x: unidecode(str(x)))  # Ensure salary data also has ASCII names
# merged_df = pd.merge(salary_df, index_df, left_on='Player', right_on='player', how='inner')

# Save or process the merged data further if needed
# print(merged_df.head())
def convert_salary(salary_str):
    if isinstance(salary_str, str):  # Only process if salary_str is a string
        # Remove '$' and ',' from the string, then convert to float
        salary_num = float(salary_str.replace('$', '').replace(',', ''))
        return salary_num
    else:
        return salary_str  # If not a string, return as-is (e.g., NaN)


salary=pd.read_csv('nba_salaries_raw.csv')
salary.to_csv('nba_salaries_raw.csv',index=False)

salary=salary[~salary.Salary.isna()]
salary=salary[~salary.Player.isna()]


index_df['player'] = index_df['player'].apply(remove_acronyms)

# Clean 'Player' names in the salary data file
salary.rename(columns={'Player':'player','Team':'team','Year':'year','Salary':'salary'},inplace=True)
salary=salary[salary.player.str.lower()!='totals']

salary['player'] = salary['player'].apply(remove_acronyms)

print(len(salary))

salary['salary'] =salary['salary'].apply(convert_salary)

salary_df=salary.merge(index_df,on=['player','year','team'],how='left')

not_found1=salary_df[salary_df.nba_id.isna()]
not_found1=not_found1[['player','year','team','salary']]
salary_df.dropna(subset='nba_id',inplace=True)
not_found1.sort_values(by='salary')

index_small=index_df[['player','year','nba_id','bref_id','url']].reset_index(drop=True)
index_small['player']=index_small['player'].str.split(' ',expand=True)[0] + ' '+index_small['player'].str.split(' ',expand=True)[1]

index_small['player'] = index_small['player'].str.replace(r'[^a-zA-Z\s]', '', regex=True).str.strip()
index_small=index_small.dropna(subset='player')
not_found1['player'] = not_found1['player'].str.replace(r'[^a-zA-Z\s]', '', regex=True).str.strip()

not_found2=not_found1.merge(index_small,on=['player','year'],how='left')
salary_df=pd.concat([salary_df,not_found2])
salary_df.dropna(subset='nba_id',inplace=True)
salary_df.drop_duplicates(inplace=True)

not_found2=not_found2[not_found2.nba_id.isna()]
not_found2.drop_duplicates(inplace=True)

not_found2=not_found2[['player','year','team','salary']]
print(len(salary_df))
salary_df

17179
15701


,player,salary,team,year,url,bref_id,nba_id
0,Moses Malone,2406000.0,ATL,1991,https://www.basketball-reference.com/players/m...,malonmo01,77449.0
1,Dominique Wilkins,2065000.0,ATL,1991,https://www.basketball-reference.com/players/w...,wilkido01,1122.0
2,Jon Koncak,1550000.0,ATL,1991,https://www.basketball-reference.com/players/k...,koncajo01,402.0
3,Doc Rivers,895000.0,ATL,1991,https://www.basketball-reference.com/players/r...,riverdo01,470.0
4,Rumeal Robinson,800000.0,ATL,1991,https://www.basketball-reference.com/players/r...,robinru01,716.0
...,...,...,...,...,...,...,...
2612,Vernon Carey,100000.0,UTA,2023,https://www.basketball-reference.com/players/c...,careyve01,1630176.0
2613,Otto Porter,6300000.0,UTA,2024,https://www.basketball-reference.com/players/p...,porteot01,203490.0
2614,Kira Lewis,5722116.0,UTA,2024,https://www.basketball-reference.com/players/l...,lewiski01,1630184.0
2617,Kevin Knox,2144320.0,UTA,2024,https://www.basketball-reference.com/players/k...,knoxke01,1628995.0


In [2]:
index_small=index_df[['player','year','nba_id','team','bref_id','url']].reset_index(drop=True)

index_small=index_small[~index_small.nba_id.isin(salary_df.nba_id.tolist())]

not_found2['last']= not_found2['player'].str.split(' ',expand=True)[1]+not_found2['team']

index_small['last']= index_small['player'].str.split(' ',expand=True)[1] +index_small['team']

index_small['last']+=index_small['player'].str[0:3]


not_found2['last']+=not_found2['player'].str[0:3]

index_small.dropna(subset='last',inplace=True)

index_small.drop(columns=['player'],inplace=True)

salary2=not_found2.merge(index_small,on=['last','year','team'])
salary2=salary2[~salary2.nba_id.isin(salary_df.nba_id.tolist())]

salary2.drop(columns='last',inplace=True)
#salary2.drop_duplicates(inplace=True)
salary_df=pd.concat([salary_df,salary2])
salary_df.sort_values(by=['team','year','salary'],inplace=True)
salary_df.to_csv('salary_backup.csv',index=False)

In [3]:
played=index_df.drop_duplicates(subset=['nba_id','year']).reset_index(drop=True)
played.drop(columns=['team'],inplace=True)

played=played[['year','nba_id']]

In [4]:
played

,year,nba_id
0,1991,76001.0
1,1991,51.0
2,1991,76009.0
3,1991,149.0
4,1991,76016.0
...,...,...
15815,2024,201152.0
15816,2024,1629027.0
15817,2024,1630209.0
15818,2024,203469.0


In [5]:
temp=salary_df[['player','year','nba_id','salary']]
temp.drop_duplicates(subset=['year','nba_id'],inplace=True)

/tmp/ipykernel_5893/2583396311.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.drop_duplicates(subset=['year','nba_id'],inplace=True)


In [13]:
to_merge

,year,nba_id,player,salary
0,1991,76001.0,Alaa Abdelnaby,395000.0
1,1991,51.0,Mahmoud Abdul-Rauf,1660000.0
2,1991,76009.0,Mark Acres,437000.0
3,1991,149.0,Michael Adams,825000.0
4,1991,76016.0,Mark Aguirre,1115000.0
...,...,...,...,...
15815,2024,201152.0,Thaddeus Young,8000000.0
15816,2024,1629027.0,Trae Young,40064220.0
15817,2024,1630209.0,Omer Yurtseven,2800000.0
15818,2024,203469.0,Cody Zeller,3196448.0


In [11]:
to_merge[to_merge.salary.isna()]

,year,nba_id,player,salary
14,1991,355.0,NaN,NaN
18,1991,76090.0,NaN,NaN
33,1991,41.0,NaN,NaN
46,1991,76320.0,NaN,NaN
71,1991,76483.0,NaN,NaN
...,...,...,...,...
15302,2024,1630527.0,NaN,NaN
15645,2024,1630590.0,NaN,NaN
15700,2024,203471.0,NaN,NaN
15717,2024,1630606.0,NaN,NaN


In [8]:
smerge=salary.merge(to_merge,how='left')
smerge.to_csv('nba_salaries_master.csv',index=False)
smerge

,player,salary,team,year,nba_id
0,Moses Malone,2406000.0,ATL,1991,77449.0
1,Dominique Wilkins,2065000.0,ATL,1991,1122.0
2,Jon Koncak,1550000.0,ATL,1991,402.0
3,Doc Rivers,895000.0,ATL,1991,470.0
4,Rumeal Robinson,800000.0,ATL,1991,716.0
...,...,...,...,...,...
17174,Kenneth Lofton,500000.0,UTA,2024,NaN
17175,Darius Bazley,500000.0,UTA,2024,NaN
17176,Jason Preston,315280.0,UTA,2024,1630554.0
17177,Josh Christopher,247720.0,UTA,2024,NaN


In [18]:
smerge[smerge.player.str.contains('Nicol')]

,player,salary,team,year,nba_id
440,Nicolas Brussino,393041.0,ATL,2018,1627852.0
1666,Nicolas Claxton,898310.0,BKN,2020,1629651.0
1683,Nicolas Claxton,1517981.0,BKN,2021,1629651.0
1711,Nicolas Claxton,1782621.0,BKN,2022,1629651.0
1725,Nicolas Claxton,9350000.0,BKN,2023,1629651.0
1746,Nicolas Claxton,9625000.0,BKN,2024,1629651.0
2119,Nicolas Batum,11860750.0,CHA,2016,201587.0
2132,Nicolas Batum,20869566.0,CHA,2017,201587.0
2152,Nicolas Batum,22434783.0,CHA,2018,201587.0
2167,Nicolas Batum,24000000.0,CHA,2019,201587.0


In [9]:
cap=pd.read_csv('cap.csv')

capmap=dict(zip(cap['year'],cap['Salary Cap']))
capmap

{2035: 364650000.0,
 2034: 331500000.0,
 2033: 301364000.0,
 2032: 273967000.0,
 2031: 249061000.0,
 2030: 226419000.0,
 2029: 205835000.0,
 2028: 187123000.0,
 2027: 170112000.0,
 2026: 154647000.0,
 2025: 140588000.0,
 2024: 136021000.0,
 2023: 123655000.0,
 2022: 112414000.0,
 2021: 109140000.0,
 2020: 109140000.0,
 2019: 101869000.0,
 2018: 99093000.0,
 2017: 94143000.0,
 2016: 70000000.0,
 2015: 63065000.0,
 2014: 58679000.0,
 2013: 58044000.0,
 2012: 58044000.0,
 2011: 58044000.0,
 2010: 57700000.0,
 2009: 58680000.0,
 2008: 55630000.0,
 2007: 53135000.0,
 2006: 49500000.0,
 2005: 43870000.0,
 2004: 43840000.0,
 2003: 40271000.0,
 2002: 42500000.0,
 2001: 35500000.0,
 2000: 34000000.0,
 1999: 30000000.0,
 1998: 26900000.0,
 1997: 24363000.0,
 1996: 23000000.0,
 1995: 15964000.0,
 1994: 15175000.0,
 1993: 14000000.0,
 1992: 12500000.0,
 1991: 11871000.0,
 1990: 9802000.0,
 1989: 7232000.0,
 1988: 6164000.0,
 1987: 4945000.0,
 1986: 4233000.0,
 1985: 3600000.0}

In [19]:
smerge['cap']=smerge['year'].map(capmap)
smerge

,player,salary,team,year,nba_id,cap
0,Moses Malone,2406000.0,ATL,1991,77449.0,11871000.0
1,Dominique Wilkins,2065000.0,ATL,1991,1122.0,11871000.0
2,Jon Koncak,1550000.0,ATL,1991,402.0,11871000.0
3,Doc Rivers,895000.0,ATL,1991,470.0,11871000.0
4,Rumeal Robinson,800000.0,ATL,1991,716.0,11871000.0
...,...,...,...,...,...,...
17174,Kenneth Lofton,500000.0,UTA,2024,NaN,136021000.0
17175,Darius Bazley,500000.0,UTA,2024,NaN,136021000.0
17176,Jason Preston,315280.0,UTA,2024,1630554.0,136021000.0
17177,Josh Christopher,247720.0,UTA,2024,NaN,136021000.0


In [22]:
smerge['cap%']=100*smerge['salary']/smerge['cap']
smerge.sort_values(by='cap%')

,player,salary,team,year,nba_id,cap,cap%
12732,Jacob Gilyard,5849.0,MEM,2023,1631367.0,123655000.0,0.004730
15447,Chance Comanche,5849.0,POR,2023,1628435.0,123655000.0,0.004730
1740,RaiQuan Gray,5849.0,BKN,2023,1630564.0,123655000.0,0.004730
12169,Mac McClung,5318.0,LAL,2022,NaN,112414000.0,0.004731
3305,Walter Tavares,6000.0,CLE,2017,NaN,94143000.0,0.006373
...,...,...,...,...,...,...,...
5895,Patrick Ewing,20500000.0,NYK,1998,121.0,26900000.0,76.208178
5863,Patrick Ewing,18724000.0,NYK,1996,121.0,23000000.0,81.408696
11694,Magic Johnson,14660000.0,LAL,1995,NaN,15964000.0,91.831621
2385,Michael Jordan,33140000.0,CHI,1998,893.0,26900000.0,123.197026


In [21]:
smerge['cap%'].max()

123.71218651233428

In [23]:
smerge.to_csv('nba_salaries.csv',index=False)